<a href="https://colab.research.google.com/github/gusmendesbh/project4-group2/blob/main/Used_Cars_Prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import statsmodels.api as sm
import cufflinks as cf
import plotly
import plotly.express as px


In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,773 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,252 kB]
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:14 htt

In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder\
    .appName("SparkSQL")\
    .config("spark.sql.debug.maxToStringFields", 2000)\
    .config("spark.driver.memory", "2g")\
    .getOrCreate()

In [4]:
# Read in data from S3 Bucket
from pyspark import SparkFiles
sales_url = "https://car-sales-data-group2.s3.amazonaws.com/vehicles.csv"
spark.sparkContext.addFile(sales_url)
car_sales_df = spark.read.csv(SparkFiles.get("vehicles.csv"), sep=",", header=True)

# Show the sales data.
car_sales_df.show()

+----------+--------------------+--------------------+--------------------+-----+----+------------+-----+---------+---------+----+--------+------------+------------+----+-----+----+----+-----------+---------+-----------+------+-----+----+----+------------+
|        id|                 url|              region|          region_url|price|year|manufacturer|model|condition|cylinders|fuel|odometer|title_status|transmission| VIN|drive|size|type|paint_color|image_url|description|county|state| lat|long|posting_date|
+----------+--------------------+--------------------+--------------------+-----+----+------------+-----+---------+---------+----+--------+------------+------------+----+-----+----+----+-----------+---------+-----------+------+-----+----+----+------------+
|7222695916|https://prescott....|            prescott|https://prescott....| 6000|null|        null| null|     null|     null|null|    null|        null|        null|null| null|null|null|       null|     null|       null|  null|  

In [5]:
car_sales_pandas = car_sales_df.toPandas()

In [6]:
car_sales_pandas.isnull().sum()

id                   0
url               9884
region            6901
region_url        6533
price             6446
year              7890
manufacturer     28937
model            17506
condition       187143
cylinders       190798
fuel             16344
odometer         17801
title_status     21618
transmission     15932
VIN             174486
drive           144178
size            319997
type            106892
paint_color     143839
image_url        13732
description      13733
county          382283
state            23077
lat              25397
long             24715
posting_date     22477
dtype: int64

In [7]:
car_sales_clean = car_sales_pandas.drop(['id', 'url', 'region_url', 'VIN', 'image_url', 'description', 'county', 'lat', 'long', 'posting_date', 'size', 'state'], axis=1)

In [8]:
car_sales_clean

,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,type,paint_color
0,prescott,6000,None,None,None,None,None,None,None,None,None,None,None,None
1,fayetteville,11900,None,None,None,None,None,None,None,None,None,None,None,None
2,florida keys,21000,None,None,None,None,None,None,None,None,None,None,None,None
3,worcester / central MA,1500,None,None,None,None,None,None,None,None,None,None,None,None
4,greensboro,4900,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441797,wyoming,23590,2019,nissan,maxima s sedan 4d,good,6 cylinders,gas,32226,clean,other,fwd,sedan,None
441798,wyoming,30590,2020,volvo,s60 t5 momentum sedan 4d,good,None,gas,12029,clean,other,fwd,sedan,red
441799,wyoming,34990,2020,cadillac,xt4 sport suv 4d,good,None,diesel,4174,clean,other,None,hatchback,white
441800,wyoming,28990,2018,lexus,es 350 sedan 4d,good,6 cylinders,gas,30112,clean,other,fwd,sedan,silver


In [9]:
car_sales_clean.isna().sum()/car_sales_clean.shape[0]*100

region           1.562012
price            1.459025
year             1.785868
manufacturer     6.549767
model            3.962408
condition       42.359021
cylinders       43.186314
fuel             3.699395
odometer         4.029180
title_status     4.893142
transmission     3.606140
drive           32.634076
type            24.194549
paint_color     32.557345
dtype: float64

In [10]:
car_sales_clean.dropna(subset=['year', 'odometer', 'manufacturer', 'model'])

,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,type,paint_color
27,auburn,33590,2014,gmc,sierra 1500 crew cab slt,good,8 cylinders,gas,57923,clean,other,None,pickup,white
28,auburn,22590,2010,chevrolet,silverado 1500,good,8 cylinders,gas,71229,clean,other,None,pickup,blue
29,auburn,39590,2020,chevrolet,silverado 1500 crew,good,8 cylinders,gas,19160,clean,other,None,pickup,red
30,auburn,30990,2017,toyota,tundra double cab sr,good,8 cylinders,gas,41124,clean,other,None,pickup,red
31,auburn,15000,2013,ford,f-150 xlt,excellent,6 cylinders,gas,128000,clean,automatic,rwd,truck,black
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441797,wyoming,23590,2019,nissan,maxima s sedan 4d,good,6 cylinders,gas,32226,clean,other,fwd,sedan,None
441798,wyoming,30590,2020,volvo,s60 t5 momentum sedan 4d,good,None,gas,12029,clean,other,fwd,sedan,red
441799,wyoming,34990,2020,cadillac,xt4 sport suv 4d,good,None,diesel,4174,clean,other,None,hatchback,white
441800,wyoming,28990,2018,lexus,es 350 sedan 4d,good,6 cylinders,gas,30112,clean,other,fwd,sedan,silver


In [11]:
car_sales_clean.fillna('unknown', inplace=True)

In [12]:
car_sales_clean.duplicated().sum()

65503

In [13]:
car_sales_clean = car_sales_clean.drop_duplicates()

In [14]:
car_sales_clean.shape

(376299, 14)

In [15]:
manufacturer_values = car_sales_clean['manufacturer'].value_counts()

In [16]:
print(manufacturer_values)

ford                        61159
chevrolet                   47409
toyota                      29609
unknown                     20003
honda                       18370
                            ...  
2021-04-28T11:07:47-0400        1
2021-04-28T11:07:31-0400        1
2021-04-28T11:07:14-0400        1
2021-04-28T11:06:58-0400        1
2021-05-03T11:06:27-0700        1
Name: manufacturer, Length: 871, dtype: int64


In [17]:
car_sales_clean['model'].value_counts()

unknown                    10285
f-150                       6554
silverado 1500              4242
1500                        3375
camry                       2558
                           ...  
94 Suburbans Diesels           1
1950 Plymouth Coupe            1
yukon xl denali 1 owner        1
InfinityM37XS                  1
Paige Glenbrook Touring        1
Name: model, Length: 30052, dtype: int64

In [23]:
manufactures_list = ['ford','chevrolet','toyota','honda','jeep','nissan','ram','gmc','bmw','dodge','mercedes-benz','hyundai','volkswagen','subaru','lexus','kia','audi','cadillac','acura','chrysler','buick','mazda','infinity','lincoln','volvo','mitsubishi','mini','pontiac','jaguar','rover','porsche','mercury','saturn','alfa-romeo','tesla','fiat','harley-davidson','ferrari','datsun','aston-martin','land rover']
car_sales_clean_manufactures = car_sales_clean.loc[car_sales_clean['manufacturer'].isin(manufactures_list)].copy()

In [28]:
manufacturer_values = car_sales_clean_manufactures['manufacturer'].value_counts()
print(manufacturer_values)

ford               61159
chevrolet          47409
toyota             29609
honda              18370
jeep               16217
nissan             16108
ram                15573
gmc                14603
bmw                12980
dodge              11476
mercedes-benz      10402
hyundai             8586
volkswagen          8434
subaru              8290
lexus               7241
kia                 7077
audi                6894
cadillac            6052
acura               5525
chrysler            5204
buick               4842
mazda               4782
lincoln             3829
volvo               3016
mitsubishi          2900
mini                2111
pontiac             2064
rover               1861
jaguar              1841
porsche             1209
mercury             1024
saturn               956
alfa-romeo           864
tesla                828
fiat                 706
harley-davidson      133
ferrari               84
datsun                62
aston-martin          22
land rover            15


In [33]:
models_values = car_sales_clean_manufactures['model'].value_counts()
models_values[0:50]

f-150                 6554
unknown               4589
silverado 1500        4242
1500                  3374
camry                 2558
silverado             2526
accord                2492
civic                 2383
escape                2275
2500                  2255
wrangler              2209
altima                2205
explorer              2097
tacoma                2086
grand cherokee        2050
mustang               1904
corolla               1794
equinox               1653
cr-v                  1617
fusion                1600
corvette              1527
focus                 1512
tahoe                 1435
rav4                  1387
malibu                1386
impala                1374
sierra 1500           1366
grand caravan         1354
sonata                1350
silverado 2500hd      1315
outback               1290
3500                  1286
cruze                 1271
odyssey               1262
charger               1259
prius                 1232
f-250                 1221
j

In [43]:
from ast import mod
model_values = car_sales_clean["model"].value_counts()
filtered_model_values = model_values[model_values > 1]
filtered_model_values
filtered_model_names = filtered_model_values[:51].index
models_to_keep = filtered_model_names.tolist()
models_to_keep.pop(0)
models_to_keep


['f-150',
 'silverado 1500',
 '1500',
 'camry',
 'silverado',
 'accord',
 'civic',
 'escape',
 '2500',
 'wrangler',
 'altima',
 'explorer',
 'tacoma',
 'grand cherokee',
 'mustang',
 'corolla',
 'equinox',
 'cr-v',
 'fusion',
 'corvette',
 'focus',
 'tahoe',
 'rav4',
 'malibu',
 'impala',
 'sierra 1500',
 'grand caravan',
 'sonata',
 'silverado 2500hd',
 'outback',
 '3500',
 'cruze',
 'odyssey',
 'charger',
 'prius',
 'f-250',
 'jetta',
 'edge',
 'tundra',
 'elantra',
 'forester',
 'sentra',
 'rogue',
 'sierra',
 'wrangler unlimited',
 'camaro',
 'f150',
 'pilot',
 'suburban',
 '4runner']

In [72]:
car_sales_clean_manufactures_models = car_sales_clean_manufactures.loc[car_sales_clean_manufactures['model'].isin(models_to_keep)].copy()
car_sales_clean_manufactures_models

,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,type,paint_color
28,auburn,22590,2010,chevrolet,silverado 1500,good,8 cylinders,gas,71229,clean,other,unknown,pickup,blue
34,auburn,35000,2019,toyota,tacoma,excellent,6 cylinders,gas,43000,clean,automatic,4wd,truck,grey
59,auburn,14000,2012,honda,odyssey,excellent,6 cylinders,gas,95000,clean,automatic,fwd,mini-van,silver
159,auburn,19000,2017,subaru,forester,like new,4 cylinders,gas,22700,lien,automatic,unknown,SUV,red
170,birmingham,13950,2011,toyota,tacoma,good,4 cylinders,gas,151060,clean,automatic,rwd,truck,silver
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441758,wyoming,0,2018,ram,2500,excellent,6 cylinders,diesel,20492,clean,automatic,4wd,truck,white
441770,wyoming,4277,2005,honda,civic,unknown,unknown,gas,236787,clean,automatic,unknown,coupe,unknown
441773,wyoming,6577,2013,hyundai,elantra,unknown,unknown,gas,169396,clean,manual,unknown,sedan,blue
441774,wyoming,6977,2011,honda,civic,unknown,unknown,gas,112396,clean,automatic,unknown,sedan,white


In [73]:
car_sales_clean_manufactures_models.loc[car_sales_clean_manufactures_models['model'] == 'f150', 'model'] = 'f-150'
car_sales_clean_manufactures_models.loc[car_sales_clean_manufactures_models['model'] == 'silverado 1500', 'model'] = 'silverado'
car_sales_clean_manufactures_models.loc[car_sales_clean_manufactures_models['model'] == 'silverado 2500hd', 'model'] = 'silverado'
car_sales_clean_manufactures_models.loc[car_sales_clean_manufactures_models['model'] == 'sierra 1500', 'model'] = 'sierra'
car_sales_clean_manufactures_models.loc[car_sales_clean_manufactures_models['model'] == 'wrangler unlimited', 'model'] = 'wrangler'


In [74]:
car_sales_clean_manufactures_models['model'].value_counts()

silverado         8083
f-150             7496
1500              3374
wrangler          3250
camry             2558
accord            2492
sierra            2420
civic             2383
escape            2275
2500              2255
altima            2205
explorer          2097
tacoma            2086
grand cherokee    2050
mustang           1904
corolla           1794
equinox           1653
cr-v              1617
fusion            1600
corvette          1527
focus             1512
tahoe             1435
rav4              1387
malibu            1386
impala            1374
grand caravan     1354
sonata            1350
outback           1290
3500              1286
cruze             1271
odyssey           1262
charger           1259
prius             1232
f-250             1221
jetta             1205
edge              1204
tundra            1195
elantra           1157
forester          1128
sentra            1079
rogue             1062
camaro            1038
pilot              941
suburban   

In [75]:
car_sales_clean_manufactures_models.describe()

,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,type,paint_color
count,85607,85607,85607,85607,85607,85607,85607,85607,85607,85607,85607,85607,85607,85607
unique,404,7993,76,13,45,7,9,6,38884,7,4,4,14,13
top,des moines,0,2014,ford,silverado,unknown,unknown,gas,unknown,clean,automatic,4wd,unknown,unknown
freq,858,7807,6795,19312,8083,44261,35327,76844,1256,81800,78476,32369,21431,26356


In [76]:
car_sales_clean_manufactures_models.isnull().sum()

region          0
price           0
year            0
manufacturer    0
model           0
condition       0
cylinders       0
fuel            0
odometer        0
title_status    0
transmission    0
drive           0
type            0
paint_color     0
dtype: int64

In [77]:
car_sales_clean_manufactures_models.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85607 entries, 28 to 441775
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   region        85607 non-null  object
 1   price         85607 non-null  object
 2   year          85607 non-null  object
 3   manufacturer  85607 non-null  object
 4   model         85607 non-null  object
 5   condition     85607 non-null  object
 6   cylinders     85607 non-null  object
 7   fuel          85607 non-null  object
 8   odometer      85607 non-null  object
 9   title_status  85607 non-null  object
 10  transmission  85607 non-null  object
 11  drive         85607 non-null  object
 12  type          85607 non-null  object
 13  paint_color   85607 non-null  object
dtypes: object(14)
memory usage: 9.8+ MB


In [82]:
car_sales_clean_manufactures_models = car_sales_clean_manufactures_models.replace('unknown', np.nan)


In [83]:
car_sales_clean_manufactures_models = car_sales_clean_manufactures_models.replace('other', np.nan)


In [84]:
car_sales_clean_manufactures_models

,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,type,paint_color
28,auburn,22590,2010,chevrolet,silverado,good,8 cylinders,gas,71229,clean,NaN,NaN,pickup,blue
34,auburn,35000,2019,toyota,tacoma,excellent,6 cylinders,gas,43000,clean,automatic,4wd,truck,grey
59,auburn,14000,2012,honda,odyssey,excellent,6 cylinders,gas,95000,clean,automatic,fwd,mini-van,silver
159,auburn,19000,2017,subaru,forester,like new,4 cylinders,gas,22700,lien,automatic,NaN,SUV,red
170,birmingham,13950,2011,toyota,tacoma,good,4 cylinders,gas,151060,clean,automatic,rwd,truck,silver
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441758,wyoming,0,2018,ram,2500,excellent,6 cylinders,diesel,20492,clean,automatic,4wd,truck,white
441770,wyoming,4277,2005,honda,civic,NaN,NaN,gas,236787,clean,automatic,NaN,coupe,NaN
441773,wyoming,6577,2013,hyundai,elantra,NaN,NaN,gas,169396,clean,manual,NaN,sedan,blue
441774,wyoming,6977,2011,honda,civic,NaN,NaN,gas,112396,clean,automatic,NaN,sedan,white
